## Introduction/Business Problem

Climate events are causing accidents on the roads of Seattle. Accidents can have severe mental and physical impacts on those directly involved and also reduce the efficiency and effectiveness of transport infrastructure. 

We want to understand if it is possible to use data from past accidents with corresponding weather data to predict the severity of these accidents.   

If we find that that there is a relationship between weather and accidents then we can use this data to educate road users and / or to put weather specific safety measures in place. We would thus hope to reduce the direct and indirect costs of accidents.

Conclusions drawn from this data set may also be applicable to other other parts of the country / world. We can assess the model to see if this is the case.




## Data

We have a dataset from the Seattle Department of Transport that shows all collisions between 2004 and present. The data is provided by Seattle Police Department and recorded by Traffic Records.  

The data in the dataset is updated weekly and currently contains over 194,000 records.

Our target variable is the severity of the accident which has 5 levels. 

The dataset contains data on the location, weather, time and circumstance of the accidents. 

We will use weather data fields as the independent variables and severity score as the dependent variable. 

The data we have available to understand the weather is as follows

ROADCOND: A description of the weather conditions during
the time of the collision.

ROADCOND - explains if the road is wet, dry, icey, or has standing water.
LIGHTCOND - explains the lighting at the time of the accident.
WEATHER - a description of the weather at the time of the accident, there are currently 10 options. There are rows that will need removing due to missing data.  

An example record shows the following:

At 5TH AVE NE AND NE 103RD ST there was a severity 2 accident where two motor vehicles collided with a front end collision during overcast weather, where the road was wet and lighting was daylight. 

In [4]:
#importing modules required
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

#defining the data source
data = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv"




In [5]:
#creating a pandas dataframe with the data set and taking a look at the data

pd.set_option('display.max_columns', None)
completeDf = pd.read_csv(data)
completeDf.head(5)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [6]:
# taking a look at the data
completeDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
SEVERITYCODE      194673 non-null int64
X                 189339 non-null float64
Y                 189339 non-null float64
OBJECTID          194673 non-null int64
INCKEY            194673 non-null int64
COLDETKEY         194673 non-null int64
REPORTNO          194673 non-null object
STATUS            194673 non-null object
ADDRTYPE          192747 non-null object
INTKEY            65070 non-null float64
LOCATION          191996 non-null object
EXCEPTRSNCODE     84811 non-null object
EXCEPTRSNDESC     5638 non-null object
SEVERITYCODE.1    194673 non-null int64
SEVERITYDESC      194673 non-null object
COLLISIONTYPE     189769 non-null object
PERSONCOUNT       194673 non-null int64
PEDCOUNT          194673 non-null int64
PEDCYLCOUNT       194673 non-null int64
VEHCOUNT          194673 non-null int64
INCDATE           194673 non-null object
INCDTTM           194673 non-null obje

In [12]:
#we will create a new df that contains just the columns we think we need

df = completeDf[["WEATHER", "ROADCOND","LIGHTCOND", "SEVERITYCODE"]]
df.head()


,WEATHER,ROADCOND,LIGHTCOND,SEVERITYCODE
0,Overcast,Wet,Daylight,2
1,Raining,Wet,Dark - Street Lights On,1
2,Overcast,Dry,Daylight,1
3,Clear,Dry,Daylight,1
4,Raining,Wet,Daylight,2


In [13]:
#looking at the info of the new df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 4 columns):
WEATHER         189592 non-null object
ROADCOND        189661 non-null object
LIGHTCOND       189503 non-null object
SEVERITYCODE    194673 non-null int64
dtypes: int64(1), object(3)
memory usage: 5.9+ MB


In [26]:
#finding out how many null rows we have in each column

df.isnull().sum(axis = 0)

WEATHER         5081
ROADCOND        5012
LIGHTCOND       5170
SEVERITYCODE       0
dtype: int64

In [31]:
#checking the % of the data that contains missing data in at least one of its rows.
df.isnull().sum().sum() / 194673

0.07840327112645308

In [87]:
#here we are creating a new df by filtering to show the columns where all the independent variables are NaN

#the columns we want to filter for
NANcols = ["WEATHER", "ROADCOND", "LIGHTCOND"]
filteredDf = df[df[NANcols].isna().all(1)]

#taking a look at the result
filteredDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4947 entries, 15 to 194655
Data columns (total 4 columns):
WEATHER         0 non-null object
ROADCOND        0 non-null object
LIGHTCOND       0 non-null object
SEVERITYCODE    4947 non-null int64
dtypes: int64(1), object(3)
memory usage: 193.2+ KB


In [88]:
#we are dropping the rows that dont have at least 2 non NaN values
withoutNullDf = df.dropna(thresh=2)

#taking a look at the remaining data
withoutNullDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189726 entries, 0 to 194672
Data columns (total 4 columns):
WEATHER         189592 non-null object
ROADCOND        189661 non-null object
LIGHTCOND       189503 non-null object
SEVERITYCODE    189726 non-null int64
dtypes: int64(1), object(3)
memory usage: 7.2+ MB


In [93]:
#taking a look at the rows that have single NaN values
NaNrows = withoutNullDf.isnull().sum(axis = 0).sum()

In [99]:
#checking the % of NaN rows 
NaNrows / withoutNullDf["SEVERITYCODE"].count()*100

0.222426024899065

In [104]:
df_coll = withoutNullDf.dropna()

In [109]:
df_coll.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189337 entries, 0 to 194672
Data columns (total 4 columns):
WEATHER         189337 non-null object
ROADCOND        189337 non-null object
LIGHTCOND       189337 non-null object
SEVERITYCODE    189337 non-null int64
dtypes: int64(1), object(3)
memory usage: 7.2+ MB


In [112]:
df_coll = df_coll.reset_index()
df_coll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189337 entries, 0 to 189336
Data columns (total 5 columns):
index           189337 non-null int64
WEATHER         189337 non-null object
ROADCOND        189337 non-null object
LIGHTCOND       189337 non-null object
SEVERITYCODE    189337 non-null int64
dtypes: int64(2), object(3)
memory usage: 7.2+ MB
